# PRD: Market Research for Coffee Capsules v2.0

## Objective  
Assess the market opportunity for coffee capsules—an emerging coffee product format—by analyzing market size, key players, target audience, and competitive positioning versus other coffee types.

You need have a plan for the research. It includes:
1. Search related companies and products.
2. Summary of findings.
3. Generate the report as a html file.

## Scope  
The research must include:

- **Market Size**: Current global/regional market value and volume, plus 3–5 year growth projections.  
- **Key Players**: Major brands (e.g., Nespresso, Keurig, Lavazza), their market share, pricing, and distribution.  


## Requirements  
- Use data from the last 18 months  
- Highlight sustainability aspects (e.g., recyclability, consumer concerns)  
- Focus on actionable insights for product and marketing teams

Let's start.
First step is to prepare the MCP tool.

In [1]:
# Lets prepare the tool at first

from course_code.common.utils import load_env, stdio_mcp
import os

from agentscope.agent import ReActAgent
from agentscope.memory import InMemoryMemory
from agentscope.mcp import  StdIOStatefulClient
from agentscope.tool import (
    Toolkit,
)
# prepare the env
# please ensure that you have set the environment variables in .env file
load_env()

# prepare the MCP client of WebSearch
stdio_json={
    "mcp-server-firecrawl": {
      "args": [
        "-y",
        "firecrawl-mcp"
      ],
      "command": "npx",
      "env": {
        "FIRECRAWL_API_KEY": os.environ.get("FIRECRAWL_API_KEY","")
      }
    }
  }

search_mcp_client = StdIOStatefulClient(
    **stdio_mcp(stdio_json)
)
await search_mcp_client.connect()

2025-11-04 17:14:45,319 | INFO    | _stateful_client_base:connect:66 - MCP client connected.


Second step is build the plan.

In [2]:
from agentscope.plan import PlanNotebook, SubTask

plan_notebook = PlanNotebook()
await plan_notebook.create_plan(
    name="Market Research",
    description="Take a deep research on coffee capsules",
    expected_outcome="A markdown report to figure the market size and key players",
    subtasks=[
        SubTask(
            name="Search related companies and products.",
            description=(
                "Search related companies and products from 2024 to 2025"
            ),
            expected_outcome="Content in Markdown",
        ),
        SubTask(
            name="Summary of findings.",
            description="Summarize the content, find out the market size and key players",
            expected_outcome="Content Markdown",
        ),
        SubTask(
            name="Generate the report.",
            description=(
                "Generate the full report with three sections: 1. Intro coffee capsules; 2. Market size; 3. key players"
            ),
            expected_outcome="Final report in html",
        )
    ],
)

ToolResponse(content=[{'type': 'text', 'text': "Plan 'Market Research' created successfully."}], metadata=None, stream=False, is_last=True, is_interrupted=False, id='2025-11-04 17:14:45.329_8a9efe')

Third step is to build the agent.

In [3]:
# construct the prompt according to the PRD.
system_prompt="""You are a senior product researcher"""
prompt ="""
"""

In [7]:
from course_code.common.utils import markdown_to_html
from agentscope.formatter import OpenAIChatFormatter, DashScopeChatFormatter
from agentscope.message import Msg
from agentscope.model import OpenAIChatModel, DashScopeChatModel
from course_code.common.constants import MODELSCOPE_ENDPOINT,QWEN_MODEL_NAME
# construct the toolkit
toolkit = Toolkit()
# register the search mcp client in toolkit
await toolkit.register_mcp_client(
    search_mcp_client
)
toolkit.register_tool_function(
    markdown_to_html,
    func_description="the tool to convert the markdown string into a html file"
    
)

# build the  agent in the ReAct 
agent = ReActAgent(
    # The name of the Agent
    name="MiMCoffeeResearcher",
    # The system prompt we prepared before.
    sys_prompt=system_prompt,
    # An OpenAI-compatible model
    model=DashScopeChatModel(
        # The API key of the ModelScope
        api_key=os.environ.get("DASHSCOPE_API_KEY"),
        # The model name
        model_name="qwen-max",
        # The args to declare the Endpoint

        stream=False,
    ),
    formatter=DashScopeChatFormatter(),
    plan_notebook=plan_notebook,
    toolkit=toolkit,
    memory=InMemoryMemory(),
)

# run the agent
reply = await agent( Msg("User",prompt,role="user"))

2025-11-04 17:17:31,984 | INFO    | _toolkit:register_mcp_client:716 - Registered 6 tool functions from MCP: firecrawl_scrape, firecrawl_map, firecrawl_search, firecrawl_crawl, firecrawl_check_crawl_status, firecrawl_extract.


MiMCoffeeResearcher: To proceed with the subtask "Search related companies and products" as part of our market research on coffee capsules, I will perform a web search to find relevant information from 2024 to 2025. This will help us identify key players and their products in the market. Let's start by searching for the latest information on coffee capsule companies and products within that timeframe.
{
    "type": "tool_use",
    "name": "firecrawl_search",
    "input": {
        "query": "coffee capsule companies and products 2024 to 2025",
        "limit": 10,
        "sources": [
            {
                "type": "web"
            }
        ]
    },
    "id": "call_352c8835942a4c7f90fc40"
}

system: {
    "type": "tool_result",
    "id": "call_352c8835942a4c7f90fc40",
    "name": "firecrawl_search",
    "output": [
        {
            "type": "text",
            "text": "{\n  \"web\": [\n    {\n      \"url\": \"https://www.intellectualmarketinsights.com/blogs/leading-companie

In [10]:
print(reply.content)

I've encountered a technical issue while trying to formally finish the subtask "Search related companies and products." However, I have already provided the summary of our findings. Let's proceed to the next subtask, which is to summarize the findings and generate the report. We can consider the current subtask as completed based on the information we have.

### Summary of Findings:

- **Key Players**: The major players in the coffee capsule market include Starbucks Corporation, Nestlé (Nescafé & Nespresso), J.M. Smucker Company, Luigi Lavazza SpA, Jacobs Douwe Egberts B.V., and Gloria Jean's Gourmet Coffees Corp. Other significant companies are Keurig Dr Pepper, Dualit, and Philips.
- **Market Size and Growth**: In 2024, the coffee capsule market was valued at $14.99 billion, with an expected growth to $16.42 billion in 2025. By 2029, it is projected to reach $22.56 billion, with a CAGR of 8.4% between 2024 and 2029. Additionally, the market is forecasted to be worth USD 270.8 million